In [7]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from google.cloud import storage
import os
import pandas as pd

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "development-416403-b306695feb67.json"

In [24]:
# open file and 
start_time = time.time()
lemmatizer = WordNetLemmatizer()
files = open('llm.txt', 'r')

text = files.read()

# chunk to word
words = [word.lower() for word in text.split() if word.isalpha()]

# remove the stopwords
stop_words = set(stopwords.words('english'))

without_stop_word_file = [word for word in words if word not in stop_words]


#  store the most commom word in list
most_common_word_list = []

for most_common_word, _ in FreqDist(without_stop_word_file).most_common(10):
    most_common_word_list.append(most_common_word)


print("most_common_words:", most_common_word_list)
    # vectorize the list of the most common word
tfid = TfidfVectorizer()
text_vector =tfid.fit_transform(most_common_word_list)
# print(tfid.get_feature_names_out())
print(text_vector.todense())

# time taken for execution
end_time = time.time()
elapsed_time = end_time - start_time
print("time taken:",elapsed_time)
# time taken: 0.006171464920043945

most_common_words: ['language', 'llms', 'clinical', 'patient', 'documentation', 'data', 'text', 'clinicians', 'including', 'content']
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
time taken: 0.2108461856842041


In [ ]:
pip list


In [8]:
def clean_data (text):


    # lemmatizer = WordNetLemmatizer(text)
    # chunk to word
    words = [word.lower() for word in text.split() if word.isalpha()]

    # remove the stopwords
    stop_words = set(stopwords.words('english'))

    without_stop_word_file = [word for word in words if word not in stop_words]


    #  store the most commom word in list
    most_common_word_list = []

    for most_common_word, _ in FreqDist(without_stop_word_file).most_common(10):
        most_common_word_list.append(most_common_word)
    
    # print("most_common_words:", most_common_word_list)
    return most_common_word_list

        # vectorize the list of the most common word
    # tfid = TfidfVectorizer()
    # text_vector =tfid.fit_transform(most_common_word_list)
    # print(tfid.get_feature_names_out())
    # print(text_vector.todense())
    # time taken for execution


read the file from bucket
split it to word
remove stopwords
lemmatize
get most common word 
store it in form of a dic the file name as key and common words as value

In [11]:
# dic for storing most common words in each file
start_time_n = time.time()
dic_most_common_word = dict()

# fetch file from the bucket
def fetch_files ():
    star_fn = time.time()
    storage_client = storage.Client()
    bucket_name = "development_awarri"
    prefix = "read/english/src-docs"
    source_bucket = storage_client.bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=None)
    for blob in blobs:
        file = blob.name
        dic_key = file.split("/")[-1]
        blob_file = source_bucket.blob(file)
        with blob_file.open("r") as f:

            # read file by line
            text = f.read()
            
            # using function to clean and get the most common words
            common_words = clean_data(text)
        #    storeing the result into dictionary with file name as key
            dic_most_common_word[dic_key] = common_words
            # print(text)
fetch_files()   
end_time_n = time.time()
print(dic_most_common_word)
print("time taken:", end_time_n-start_time_n)

{' 200 BDC operators arrested, EFCC alleges fraud.txt': ['arrested', 'suspects', 'efcc', 'forex', 'enugu', 'one', 'financial', 'oyewale', 'state', 'charge'], ' A National Eulogy in Remembering the Fallen Hereos.txt': ['us', 'national', 'names', 'world', 'nation', 'prayers', 'many', 'god', 'nigerians', 'life'], " African Night's Entertainment.txt": ['novel', 'abu', 'child', 'kyauta', 'old', 'story', 'vengeance', 'mallam', 'different', 'make'], ' Awọn oṣiṣẹ ileewe ijọba ipinlẹ Ọyọ binu si Gomina Makinde.txt': ['n', 'ti', 'awọn', 'wọn', 'ṣe', 'ni', 'ileewe', 'oṣiṣẹ', 'ijọba', 'lawọn'], ' Did Tinubu appoint retired AIG Odumosu substantive EFCC Chairman?.txt': ['odumosu', 'efcc', 'government', 'appointed', 'retired', 'tinubu', 'social', 'media', 'official', 'statement'], ' Echoes of Tomorrow.txt': ['charlotte', 'book', 'within', 'pages', 'existed', 'forgotten', 'obscured', 'amidst', 'old', 'one'], ' Empowering Women: A Catalyst for Sustainable Development in Nigeria.txt': ['women', 'sustain

In [15]:
# Original dictionary with filenames as keys and lists of words as values
original_dict = {
    'file1.txt': ['apple', 'banana', 'orange'],
    'file2.txt': ['grape', 'pineapple', 'watermelon'],
    'file3.txt': ['cherry', 'strawberry', 'blueberry']
}

# Convert to a new dictionary where keys are filenames and values are flattened lists of words
flattened_dict = {key: words for key, words_list in original_dict.items() for words in [words_list]}

print(flattened_dict)


{'file1.txt': ['apple', 'banana', 'orange'], 'file2.txt': ['grape', 'pineapple', 'watermelon'], 'file3.txt': ['cherry', 'strawberry', 'blueberry']}


In [17]:
# df = pd.DataFrame(dic_most_common_word, orient='index')
df = pd.DataFrame.from_dict(dic_most_common_word,orient='index')

df.to_csv("most-common-words.csv")